In [2]:
%%capture
!pip install folium matplotlib mapclassify

In [3]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

fclass_to_keep = ['residential',
                  'commercial',
                  'industrial',
                  'retail',
                  'scrub',
                  'allotments',
                  'park',
                  'forest',
                  'grass',
                  'recreation_ground',
                  'nature_reserve',
                  'heath']

green_areas = ['scrub',
               'allotments',
               'park',
               'forest',
               'grass',
               'recreation_ground',
               'nature_reserve',
               'heath']

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import NILs

In [5]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


# Read OSM Landuse

In [6]:
osm = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed', 'OSM','gis_osm_landuse_a_free_1.shp'))
osm = osm[osm['fclass'].isin(fclass_to_keep)]
osm.loc[osm['fclass'].isin(green_areas), 'fclass'] = 'green_area'
osm

,osm_id,code,fclass,name,geometry
0,310001057,7217,green_area,None,"POLYGON ((9.13155 45.41577, 9.13170 45.41586, ..."
18,1265668272,7201,green_area,None,"POLYGON ((9.12686 45.42862, 9.12688 45.42893, ..."
20,244070993,7202,green_area,None,"POLYGON ((9.12462 45.43033, 9.12480 45.43039, ..."
25,746144217,7202,green_area,Parco della Vita,"POLYGON ((9.07473 45.43540, 9.07536 45.43546, ..."
27,397247322,7203,residential,Campo nomadi,"POLYGON ((9.07194 45.44195, 9.07333 45.44218, ..."
...,...,...,...,...,...
7384,438914907,7203,residential,Quartiere Vialba I,"POLYGON ((9.12810 45.51921, 9.12992 45.52014, ..."
7387,886328307,7203,residential,Campo nomadi,"POLYGON ((9.11102 45.52125, 9.11165 45.52180, ..."
7388,429005042,7204,industrial,None,"POLYGON ((9.10922 45.52387, 9.11115 45.52481, ..."
7389,429005041,7204,industrial,None,"POLYGON ((9.10861 45.52488, 9.10914 45.52513, ..."


# Overlay areas to fix intersections on NILs

In [7]:
osm_landuse = gpd.overlay(osm, nil, how='intersection')
osm_landuse.sort_values(by='ID_NIL')

,osm_id,code,fclass,name,ID_NIL,NIL,area_sqkm,Residenti,geometry
1486,564836183,7203,residential,None,1,duomo,2.341616,16608,"POLYGON ((9.19003 45.45753, 9.19014 45.45706, ..."
1479,404741750,7209,commercial,None,1,duomo,2.341616,16608,"POLYGON ((9.18526 45.45689, 9.18531 45.45689, ..."
1480,404741747,7209,commercial,None,1,duomo,2.341616,16608,"POLYGON ((9.18448 45.45693, 9.18448 45.45718, ..."
1481,396367484,7218,green_area,None,1,duomo,2.341616,16608,"POLYGON ((9.19094 45.45760, 9.19101 45.45735, ..."
1482,564836182,7203,residential,None,1,duomo,2.341616,16608,"POLYGON ((9.19089 45.45762, 9.19125 45.45806, ..."
...,...,...,...,...,...,...,...,...,...
4896,428376907,7209,commercial,None,9,porta garibaldi - porta nuova,0.785638,5921,"POLYGON ((9.19446 45.48470, 9.19455 45.48477, ..."
4960,428574717,7203,residential,None,9,porta garibaldi - porta nuova,0.785638,5921,"POLYGON ((9.19582 45.48815, 9.19581 45.48816, ..."
4897,428376922,7203,residential,None,9,porta garibaldi - porta nuova,0.785638,5921,"POLYGON ((9.19805 45.48518, 9.19804 45.48521, ..."
4891,428412146,7212,retail,None,9,porta garibaldi - porta nuova,0.785638,5921,"POLYGON ((9.19732 45.48332, 9.19809 45.48317, ..."


In [8]:
cri = osm_landuse[osm_landuse['fclass'] != 'green_area']

green_areas = osm_landuse[osm_landuse['fclass'] == 'green_area']
green_areas = green_areas.dissolve(by=['ID_NIL','NIL']).reset_index()
green_areas

,ID_NIL,NIL,geometry,osm_id,code,fclass,name,area_sqkm,Residenti
0,1,duomo,"MULTIPOLYGON (((9.17487 45.46257, 9.17490 45.4...",542621726,7218,green_area,Parco delle Basiliche,2.341616,16608
1,10,stazione centrale - ponte seveso,"MULTIPOLYGON (((9.19785 45.47826, 9.19784 45.4...",217332259,7218,green_area,Giardino delle Bambine e dei Bambini di tutto ...,1.556019,18410
2,11,isola,"MULTIPOLYGON (((9.18757 45.48605, 9.18777 45.4...",10036938,7202,green_area,Parco Biblioteca degli Alberi,1.322887,22641
3,12,maciachini - maggiolina,"MULTIPOLYGON (((9.19071 45.49733, 9.19073 45.4...",458846143,7218,green_area,Giardino Aldo Protti,1.674918,26097
4,13,greco - segnano,"MULTIPOLYGON (((9.20744 45.49557, 9.20736 45.4...",147010065,7218,green_area,Giardino Cassina de' Pomm,1.768603,16137
...,...,...,...,...,...,...,...,...,...
83,85,parco delle abbazie,"MULTIPOLYGON (((9.18219 45.39110, 9.18219 45.3...",75113307,7202,green_area,Parco agricolo del Ticinello,13.733841,364
84,86,parco dei navigli,"MULTIPOLYGON (((9.13320 45.41434, 9.13315 45.4...",310001057,7217,green_area,Parco agricolo Sud Milano,3.623149,354
85,87,assiano,"MULTIPOLYGON (((9.05616 45.43815, 9.05610 45.4...",746144217,7202,green_area,Parco della Vita,5.840942,211
86,88,parco bosco in citta',"MULTIPOLYGON (((9.07244 45.47851, 9.07243 45.4...",1108943838,7201,green_area,Orti Barocco,7.834006,690


In [9]:
osm_landuse = pd.concat([green_areas, cri])
osm_landuse = osm_landuse.to_crs(epsg=32632)
osm_landuse['fclass_area_sqkm'] = osm_landuse.area / 10**6
osm_landuse = osm_landuse.to_crs(epsg=4326)
osm_landuse

,ID_NIL,NIL,geometry,osm_id,code,fclass,name,area_sqkm,Residenti,fclass_area_sqkm
0,1,duomo,"MULTIPOLYGON (((9.17487 45.46257, 9.17490 45.4...",542621726,7218,green_area,Parco delle Basiliche,2.341616,16608,0.131575
1,10,stazione centrale - ponte seveso,"MULTIPOLYGON (((9.19785 45.47826, 9.19784 45.4...",217332259,7218,green_area,Giardino delle Bambine e dei Bambini di tutto ...,1.556019,18410,0.020215
2,11,isola,"MULTIPOLYGON (((9.18757 45.48605, 9.18777 45.4...",10036938,7202,green_area,Parco Biblioteca degli Alberi,1.322887,22641,0.086662
3,12,maciachini - maggiolina,"MULTIPOLYGON (((9.19071 45.49733, 9.19073 45.4...",458846143,7218,green_area,Giardino Aldo Protti,1.674918,26097,0.158593
4,13,greco - segnano,"MULTIPOLYGON (((9.20744 45.49557, 9.20736 45.4...",147010065,7218,green_area,Giardino Cassina de' Pomm,1.768603,16137,0.105546
...,...,...,...,...,...,...,...,...,...,...
7199,73,cascina merlata,"POLYGON ((9.09512 45.51928, 9.09519 45.51938, ...",193460097,7204,industrial,Poste Italiane,1.730640,943,0.056421
7201,73,cascina merlata,"POLYGON ((9.09444 45.52279, 9.09451 45.52279, ...",4816893,7209,commercial,MIND - Milano Innovation District,1.730640,943,0.782963
7207,73,cascina merlata,"MULTIPOLYGON (((9.10750 45.52242, 9.10740 45.5...",429005636,7204,industrial,None,1.730640,943,0.024645
7209,73,cascina merlata,"MULTIPOLYGON (((9.09812 45.52925, 9.09808 45.5...",12037463,7204,industrial,None,1.730640,943,0.000366




# Groupby

In [10]:
landuse_grouped = osm_landuse.groupby(['ID_NIL','NIL','fclass','area_sqkm','Residenti'])['fclass_area_sqkm'].sum().reset_index()
landuse_grouped

,ID_NIL,NIL,fclass,area_sqkm,Residenti,fclass_area_sqkm
0,1,duomo,commercial,2.341616,16608,0.028222
1,1,duomo,green_area,2.341616,16608,0.131575
2,1,duomo,residential,2.341616,16608,0.222154
3,1,duomo,retail,2.341616,16608,0.000063
4,10,stazione centrale - ponte seveso,commercial,1.556019,18410,0.120899
...,...,...,...,...,...,...
358,88,parco bosco in citta',retail,7.834006,690,0.005706
359,9,porta garibaldi - porta nuova,commercial,0.785638,5921,0.177915
360,9,porta garibaldi - porta nuova,green_area,0.785638,5921,0.095498
361,9,porta garibaldi - porta nuova,residential,0.785638,5921,0.340262


# Create per sqkm and per_person columns

In [11]:
landuse_grouped['landuse_ratio'] = landuse_grouped['fclass_area_sqkm'] / landuse_grouped['area_sqkm']
landuse_grouped['landuse_per_person'] = landuse_grouped['fclass_area_sqkm'] / landuse_grouped['Residenti']
landuse_grouped.sort_values(by=['landuse_ratio'], ascending=False)

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,ID_NIL,NIL,fclass,area_sqkm,Residenti,fclass_area_sqkm,landuse_ratio,landuse_per_person
347,86,parco dei navigli,green_area,3.623149,354,3.591817,0.991352,1.014637e-02
343,85,parco delle abbazie,green_area,13.733841,364,13.348927,0.971973,3.667288e-02
339,84,parco nord,green_area,1.532331,98,1.466983,0.957354,1.496922e-02
355,88,parco bosco in citta',green_area,7.834006,690,7.402650,0.944938,1.072848e-02
350,87,assiano,green_area,5.840942,211,5.435279,0.930548,2.575962e-02
...,...,...,...,...,...,...,...,...
194,5,porta vigentina - porta lodovica,retail,1.135196,13548,0.000264,0.000233,1.950892e-08
217,56,forze armate,commercial,3.206783,25181,0.000673,0.000210,2.672486e-08
349,86,parco dei navigli,residential,3.623149,354,0.000183,0.000051,5.175502e-07
202,52,bande nere,industrial,2.663780,44048,0.000074,0.000028,1.678630e-09


# Filter for green areas

In [14]:
green_spaces = landuse_grouped[landuse_grouped['fclass']=='green_area'].drop('fclass',axis=1)
green_spaces = green_spaces.rename(columns={'fclass_area_sqkm':'green_spaces_sqkm', 'landuse_ratio':'green_spaces_ratio','landuse_per_person':'green_space_per_person'})
green_spaces

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,ID_NIL,NIL,area_sqkm,Residenti,green_spaces_sqkm,green_spaces_ratio,green_space_per_person
1,1,duomo,2.341616,16608,0.131575,0.056190,0.000008
5,10,stazione centrale - ponte seveso,1.556019,18410,0.020215,0.012991,0.000001
10,11,isola,1.322887,22641,0.086662,0.065510,0.000004
15,12,maciachini - maggiolina,1.674918,26097,0.158593,0.094687,0.000006
19,13,greco - segnano,1.768603,16137,0.105546,0.059678,0.000007
...,...,...,...,...,...,...,...
343,85,parco delle abbazie,13.733841,364,13.348927,0.971973,0.036673
347,86,parco dei navigli,3.623149,354,3.591817,0.991352,0.010146
350,87,assiano,5.840942,211,5.435279,0.930548,0.025760
355,88,parco bosco in citta',7.834006,690,7.402650,0.944938,0.010728


# Export

In [15]:
#green_spaces.to_csv(os.path.join(FOLDER_PATH, 'Processed', 'green_spaces.csv'), index=False)